In [1]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('../raw_data/select_2020.csv', encoding= 'unicode_escape', sep=';')
df.head(20)

,zugnr,datum,bhf,arrTime,adelay,depTime,ddelay
0,EC 6,2019-12-15,Basel Bad Bf,1433,0,1434,0
1,EC 6,2019-12-15,Basel SBB,9999,0,1426,0
2,EC 6,2019-12-15,Bonn Hbf,1844,0,1845,0
3,EC 6,2019-12-15,Bremen Hbf,2217,2,2219,2
4,EC 6,2019-12-15,Duisburg Hbf,1945,6,1946,6
5,EC 6,2019-12-15,Düsseldorf Hbf,1930,2,1931,2
6,EC 6,2019-12-15,Essen Hbf,1958,5,2001,5
7,EC 6,2019-12-15,Freiburg(Breisgau) Hbf,1505,0,1506,0
8,EC 6,2019-12-15,Gelsenkirchen Hbf,2012,0,2013,0
9,EC 6,2019-12-15,Hamburg Dammtor,2322,2,2324,2


In [6]:
df.nunique()

zugnr       130
datum       364
bhf         129
arrTime    1420
adelay      244
depTime    1422
ddelay      245
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471584 entries, 0 to 471583
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   zugnr    471584 non-null  object
 1   datum    471584 non-null  object
 2   bhf      471584 non-null  object
 3   arrTime  471584 non-null  int64 
 4   adelay   471584 non-null  int64 
 5   depTime  471584 non-null  int64 
 6   ddelay   471584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 25.2+ MB


In [9]:
df['zugnr'] == 'EC 6'

0          True
1          True
2          True
3          True
4          True
          ...  
471579    False
471580    False
471581    False
471582    False
471583    False
Name: zugnr, Length: 471584, dtype: bool

In [18]:
ec6 = df[(df['zugnr'] == 'EC 6') & (df['bhf'] == 'Basel Bad Bf')]
ec6.shape

(304, 7)

In [19]:
ec6

,zugnr,datum,bhf,arrTime,adelay,depTime,ddelay
0,EC 6,2019-12-15,Basel Bad Bf,1433,0,1434,0
21,EC 6,2019-12-16,Basel Bad Bf,1433,1,1435,1
41,EC 6,2019-12-17,Basel Bad Bf,1433,2,1435,2
61,EC 6,2019-12-18,Basel Bad Bf,1433,2,1435,2
89,EC 6,2019-12-19,Basel Bad Bf,1433,0,1435,0
...,...,...,...,...,...,...,...
6851,EC 6,2020-12-08,Basel Bad Bf,1433,1,1435,0
6876,EC 6,2020-12-09,Basel Bad Bf,1433,1,1435,1
6902,EC 6,2020-12-10,Basel Bad Bf,1433,1,1435,1
6928,EC 6,2020-12-11,Basel Bad Bf,1433,5,1435,4


In [16]:
ec6 = df[(df['zugnr'] == 'EC 6') & (df['bhf'] == 'Basel Bad Bf') & (df['arrTime'] == 9999)]
ec6.shape

(2, 7)

In [17]:
ec6

,zugnr,datum,bhf,arrTime,adelay,depTime,ddelay
6630,EC 6,2020-11-28,Basel Bad Bf,9999,0,1435,0
6643,EC 6,2020-11-29,Basel Bad Bf,9999,0,1435,3


In [22]:
import requests

key = '7DYDYYY5GVYHQA52HXFQV5A5Y'
# lat = 38.9697 # Länge
# lon = -77.385 # Breite

# Example in Hamburg Elbbrücken
lat, lon = 53.53450, 10.02450

# Forecasting of 15 days URL
url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/" + str(lat) + "," + str(lon) + "?key=" + key

# Historical data
url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/" + str(lat) + "," + str(lon) + "/2020-10-01/2020-12-31?key=" + key

response = requests.get(url).json()
print(response)

JSONDecodeError: [Errno Expecting value] This query size (2208)  exceeds the maximum query cost for your account (1000). Please split your query into smaller queries. To increase your per query limit, you can add a credit card or subscribe to a monthly plan. : 0

In [26]:
import requests

key = '7DYDYYY5GVYHQA52HXFQV5A5Y'
# lat = 38.9697 # Länge
# lon = -77.385 # Breite

# TODO: Append to the database, the information of latitude and longitud of every station from the shp file

# Example in Hamburg Elbbrücken
lat, lon = 53.53450, 10.02450

# Forecasting of 15 days URL
url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/" + str(lat) + "," + str(lon) + "?key=" + key

start_date = '2020-10-01'
end_date = '2020-10-02'

# TODO: Build for loop to get daily data since the maximum quota for every query is reached
# TODO: Will I be blocked for making 365 * 2 requests in a row?
# TODO: The limit of queries' cost is 1000 per day. I already did 7 queries for a cost of 148.
#       That means that each query costs 21.14, so for a daily resolution, only for a year,
#       we would spend 7717.12 wich is equal to 7.7 days downloading data for just one year.
#       We could create "fake emails" to get a different key for each email so we can download
#       all the info in just one day.

# Historical data
url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/" + str(lat) + "," + str(lon) + "/" + start_date + "/" + end_date + "?key=" + key + "&unitGroup=metric"
print(url)

# TODO: Store every response in a dictionary or a DataFrame, or only de desire data to save memory
# TODO: Decide if we work with the average data for the whole day, or with hour resolution

response = requests.get(url).json()
print(response)

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/53.5345,10.0245/2020-10-01/2020-10-02?key=7DYDYYY5GVYHQA52HXFQV5A5Y
{'queryCost': 48, 'latitude': 53.5345, 'longitude': 10.0245, 'resolvedAddress': '53.5345,10.0245', 'address': '53.5345,10.0245', 'timezone': 'Europe/Berlin', 'tzoffset': 2.0, 'days': [{'datetime': '2020-10-01', 'datetimeEpoch': 1601503200, 'tempmax': 64.4, 'tempmin': 49.5, 'temp': 56.3, 'feelslikemax': 64.4, 'feelslikemin': 45.8, 'feelslike': 55.9, 'dew': 49.8, 'humidity': 80.1, 'precip': 0.0, 'precipprob': 0.0, 'precipcover': 0.0, 'preciptype': None, 'snow': None, 'snowdepth': None, 'windgust': None, 'windspeed': 17.4, 'winddir': 115.5, 'pressure': 1006.5, 'cloudcover': 31.5, 'visibility': 16.1, 'solarradiation': 90.9, 'solarenergy': 7.9, 'uvindex': 5.0, 'sunrise': '07:22:06', 'sunriseEpoch': 1601529726, 'sunset': '18:55:47', 'sunsetEpoch': 1601571347, 'moonphase': 0.5, 'conditions': 'Partially cloudy', 'description': 'Becoming cloudy 

In [27]:
# Import Meteostat library and dependencies
from datetime import datetime
from meteostat import Hourly

# Set time period
start = datetime(2018, 1, 1)
end = datetime(2018, 12, 31, 23, 59)

# Get hourly data
Hourly()
data = Hourly('72219', start, end)
data = data.fetch()

# Print DataFrame
print(data)

                     temp  dwpt  rhum  prcp  snow   wdir  wspd  wpgt    pres  \
time                                                                           
2018-01-01 00:00:00   0.6  -5.6  63.0   NaN   NaN  340.0  18.4   NaN  1024.6   
2018-01-01 01:00:00  -1.1  -6.6  66.0   0.0   NaN  320.0  16.6   NaN  1025.4   
2018-01-01 02:00:00  -1.1  -9.5  53.0   0.0   NaN  330.0  22.3   NaN  1025.8   
2018-01-01 03:00:00  -1.1 -10.0  51.0   0.0   NaN  330.0  25.9   NaN  1025.6   
2018-01-01 04:00:00  -1.7 -10.5  51.0   0.0   NaN  340.0  18.4   NaN  1026.1   
...                   ...   ...   ...   ...   ...    ...   ...   ...     ...   
2018-12-31 19:00:00  20.0  17.2  84.0   0.0   NaN  200.0  18.4   NaN  1016.5   
2018-12-31 20:00:00  21.1  16.7  76.0   0.0   NaN  190.0  22.3   NaN  1015.8   
2018-12-31 21:00:00  21.1  16.1  73.0   0.0   NaN  180.0  27.7   NaN  1014.7   
2018-12-31 22:00:00  21.1  15.0  68.0   0.0   NaN  190.0  27.7   NaN  1015.7   
2018-12-31 23:00:00  20.0  15.0  73.0   